In [3]:
# Celda 1: Importaciones y configuración
import hashlib
import struct
from typing import List, Dict, Tuple

class KaoruBridge:
    """
    Forensic decomposition of SHA-256 block structures.
    Extracts W-schedule values from Bitcoin block headers.
    """

    GENESIS_HASH = (
        "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
    )

    # Semantic labels for W-values
    W_LABELS = [
        "Version",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "PrevHash", "PrevHash", "PrevHash", "PrevHash",
        "Merkle", "Merkle", "Merkle", "Merkle",
        "Merkle", "Merkle", "Merkle"
    ]

    @staticmethod
    def construct_genesis_header() -> bytes:
        """Construct the exact 80-byte Genesis Block header."""
        version = struct.pack('<I', 1)
        prev_hash = bytes(32)

        # Merkle root (reversed for internal representation)
        merkle = bytes.fromhex(
            '4a5e1e4baab89f3a32518a88c31bc87f'
            '618f76673e2cc77ab2127b7afdeda33b'
        )[::-1]

        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 486604799)
        nonce = struct.pack('<I', 2083236893)

        return version + prev_hash + merkle + timestamp + bits + nonce

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        """Bitcoin's double SHA-256."""
        return hashlib.sha256(
            hashlib.sha256(data).digest()
        ).digest()

    @staticmethod
    def extract_w_values(block: bytes) -> List[int]:
        """Extract first 16 W-values from message block."""
        return [
            struct.unpack('>I', block[i:i+4])[0]
            for i in range(0, 64, 4)
        ]

    @classmethod
    def verify_genesis(cls) -> Tuple[bool, Dict]:
        """Complete Genesis Block verification."""
        header = cls.construct_genesis_header()
        hash_result = cls.double_sha256(header)[::-1].hex()
        w_values = cls.extract_w_values(header[:64])

        return hash_result == cls.GENESIS_HASH, {
            'header_hex': header.hex(),
            'header_length': len(header),
            'computed_hash': hash_result,
            'expected_hash': cls.GENESIS_HASH,
            'w_values': w_values,
            'verification': hash_result == cls.GENESIS_HASH
        }

    @classmethod
    def run_full_analysis(cls):
        """Execute complete forensic analysis with formatted output."""

        separator = "=" * 60

        print(f"\n{separator}")
        print("    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS")
        print(separator)

        # Step 1: Construct header
        print("\n[*] Constructing Genesis Block header...")
        header = cls.construct_genesis_header()
        print(f"[*] Header length: {len(header)} bytes")

        # Step 2: Apply double SHA-256
        print("[*] Applying double SHA-256...")

        # Step 3: Verify
        success, data = cls.verify_genesis()

        # Step 4: Display results
        print("\n[+] RESULTS:")

        computed = data['computed_hash']
        expected = data['expected_hash']

        print("    Computed Hash:")
        print(f"        {computed[:40]}")
        print(f"        {computed[40:]}")

        print("    Expected Hash:")
        print(f"        {expected[:40]}")
        print(f"        {expected[40:]}")

        print(f"\n    Status: {'VERIFICATION SUCCESSFUL' if success else 'VERIFICATION FAILED'}")

        # Step 5: W-Schedule extraction
        print("\n[+] W-Schedule Extraction:")
        w_values = data['w_values']

        for i, w in enumerate(w_values):
            label = cls.W_LABELS[i]
            print(f"    W[{i:2d}] = 0x{w:08x} ({label})")

        print(f"\n{separator}")
        print("    BRIDGE CROSSING COMPLETE")
        print(f"{separator}\n")

        return success, data

    @classmethod
    def create_new_blockchain(cls, original_header: bytes, original_hash: str) -> Tuple[bytes, str]:
        """Create a new blockchain starting from the genesis block."""
        new_header = original_header
        new_hash = original_hash

        # Create a new block by modifying the nonce
        for nonce in range(2**32):
            new_header = original_header[:68] + struct.pack('<I', nonce)
            new_hash = cls.double_sha256(new_header)[::-1].hex()

            # Stop if the new hash is different from the original
            if new_hash != original_hash:
                print(f"[+] New block created with nonce: {nonce}")
                return new_header, new_hash

        print("[-] Failed to create a new block.")
        return None, None

# Celda 2: Ejecución del análisis y creación de nueva cadena de bloques
if __name__ == "__main__":
    # Run full analysis on the genesis block
    success, data = KaoruBridge.run_full_analysis()

    if success:
        print("\n[*] Genesis block verification successful. Proceeding to create a new blockchain...\n")

        # Create a new blockchain starting from the genesis block
        original_header = bytes.fromhex(data['header_hex'])
        original_hash = data['computed_hash']
        new_header, new_hash = KaoruBridge.create_new_blockchain(original_header, original_hash)

        if new_header:
            print("\n[+] New blockchain created successfully!")
            print(f"    New Header: {new_header.hex()}")
            print(f"    New Hash: {new_hash}")
        else:
            print("\n[-] Failed to create a new blockchain.")
    else:
        print("\n[!] Genesis block verification failed. Cannot proceed with new blockchain creation.")


    KAORU BRIDGE: SHA-256 FORENSIC ANALYSIS

[*] Constructing Genesis Block header...
[*] Header length: 80 bytes
[*] Applying double SHA-256...

[+] RESULTS:
    Computed Hash:
        000000000019d6689c085ae165831e934ff763ae
        46a2a6c172b3f1b60a8ce26f
    Expected Hash:
        000000000019d6689c085ae165831e934ff763ae
        46a2a6c172b3f1b60a8ce26f

    Status: VERIFICATION SUCCESSFUL

[+] W-Schedule Extraction:
    W[ 0] = 0x01000000 (Version)
    W[ 1] = 0x00000000 (PrevHash)
    W[ 2] = 0x00000000 (PrevHash)
    W[ 3] = 0x00000000 (PrevHash)
    W[ 4] = 0x00000000 (PrevHash)
    W[ 5] = 0x00000000 (PrevHash)
    W[ 6] = 0x00000000 (PrevHash)
    W[ 7] = 0x00000000 (PrevHash)
    W[ 8] = 0x00000000 (PrevHash)
    W[ 9] = 0x3ba3edfd (Merkle)
    W[10] = 0x7a7b12b2 (Merkle)
    W[11] = 0x7ac72c3e (Merkle)
    W[12] = 0x67768f61 (Merkle)
    W[13] = 0x7fc81bc3 (Merkle)
    W[14] = 0x888a5132 (Merkle)
    W[15] = 0x3a9fb8aa (Merkle)

    BRIDGE CROSSING COMPLETE


[*] Genesis 